In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
####################### Check Mahim #############################################


from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [2]:
# python -c 'import tensorflow as tf; print(tf.version)'
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [3]:

img_shape=(28, 28)
encoded_dim=2
optimizer_reconst = Adam(0.001)
optimizer_discriminator = Adam(0.0001)

def _genEncoderModel(img_shape, encoded_dim):
    """ Build Encoder Model Based on Paper Configuration
    Args:
        img_shape (tuple) : shape of input image
        encoded_dim (int) : number of latent variables
    Return:
        A sequential keras model
    """
    encoder = Sequential()
    encoder.add(Flatten(input_shape=img_shape))
    encoder.add(Dense(1000, activation='relu'))
    encoder.add(Dense(1000, activation='relu'))
    encoder.add(Dense(encoded_dim))
    encoder.summary()
    return encoder

def _getDecoderModel( encoded_dim, img_shape):
    """ Build Decoder Model Based on Paper Configuration
    Args:
        encoded_dim (int) : number of latent variables
        img_shape (tuple) : shape of target images
    Return:
        A sequential keras model
    """
    latent_input = Input(shape=(encoded_dim,))
    labels_input = Input(shape=(10,))
    concated = concatenate([latent_input, labels_input])
    decoder = Sequential()
    decoder.add(Dense(1000, activation='relu', input_dim=encoded_dim+10))
    decoder.add(Dense(1000, activation='relu'))
    decoder.add(Dense(np.prod(img_shape), activation='sigmoid'))
    decoder.add(Reshape(img_shape))
    out = decoder(concated)
    decoder_model = Model([latent_input, labels_input], out)
    decoder_model.summary()
    return decoder_model

def _getDescriminator( encoded_dim):
    """ Build Descriminator Model Based on Paper Configuration
    Args:
        encoded_dim (int) : number of latent variables
    Return:
        A sequential keras model
    """
    discriminator = Sequential()
    discriminator.add(Dense(1000, activation='relu',
                            input_dim=encoded_dim))
    discriminator.add(Dense(1000, activation='relu'))
    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.summary()
    return discriminator


encoder = _genEncoderModel(img_shape, encoded_dim)
decoder = _getDecoderModel(encoded_dim, img_shape)
discriminator = _getDescriminator(encoded_dim)
img = Input(shape=img_shape)
label = Input(shape=(10,))
encoded_repr = encoder(img)
gen_img = decoder([encoded_repr, label])
autoencoder = Model([img, label], gen_img)
valid = discriminator(encoded_repr)
encoder_discriminator = Model(img, valid)
discriminator.compile(optimizer=optimizer_discriminator,
                            loss='binary_crossentropy',
                            metrics=['accuracy'])
autoencoder.compile(optimizer=optimizer_reconst,
                          loss ='mse')
for layer in discriminator.layers:
    layer.trainable = False
encoder_discriminator.compile(optimizer=optimizer_discriminator,
                                    loss='binary_crossentropy',
                                    metrics=['accuracy'])
def imagegrid( epochnumber):
    fig = plt.figure(figsize=[20, 20])
    labels = getOneHotCodes(range(10))
    styles = np.random.normal(size=(10, encoded_dim))
    k=0
    for label in labels:
        for style in styles:
            img = decoder.predict([style.reshape(-1,encoded_dim), label.reshape(-1,10)])
            img = img.reshape((28, 28))
            k = k + 1
            ax = fig.add_subplot(10, 10, k)
            ax.set_axis_off()
            ax.imshow(img, cmap="gray")
    fig.savefig("images/SAAE/" + str(epochnumber) + ".png")
    plt.show()
    plt.close(fig)

def getOneHotCodes( y_train):
    labels = np.zeros((len(y_train), 10))
    labels[range(len(y_train)), np.array(y_train).astype(int)] = 1
    return labels

def train( x_train, y_train, batch_size=32, epochs=5000, save_image_interval=50):
    labels = getOneHotCodes(y_train)
    save_interval = 50
    for epoch in range(epochs):
        #---------------Train Discriminator -------------
        # Select a random half batch of images
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train[idx]
        labels_batch = labels[idx, :]
        # Generate a half batch of new images
        latent_fake = encoder.predict(imgs)
        #gen_imgs = decoder.predict(latent_fake)
        latent_real = np.random.normal(size=(batch_size, encoded_dim))
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(latent_real, valid)
        d_loss_fake = discriminator.train_on_batch(latent_fake, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        #idx = np.random.randint(0, x_train.shape[0], batch_size)
        #imgs = x_train[idx]
        # Generator wants the discriminator to label the generated representations as valid
        valid_y = np.ones((batch_size, 1))

        # Train the autoencode reconstruction
        g_loss_reconstruction = autoencoder.train_on_batch([imgs, labels_batch], imgs)

        # Train generator
        g_logg_similarity = encoder_discriminator.train_on_batch(imgs, valid_y)
        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%] [G acc: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1],
                g_logg_similarity[1], g_loss_reconstruction))
        #if(epoch % save_interval == 0):
          #   imagegrid(epoch)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1000)              785000    
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2002      
Total params: 1,788,002
Trainable params: 1,788,002
Non-trainable params: 0
_________________________________________________________________
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)           

In [5]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32) / 255.
x_test = x_test.astype(np.float32) / 255.
train(x_train, y_train)

Streaming output truncated to the last 5000 lines.
1 [D loss: 0.689309, acc: 50.00%] [G acc: 1.000000, mse: 0.227140]
2 [D loss: 0.724541, acc: 50.00%] [G acc: 1.000000, mse: 0.211502]
3 [D loss: 0.714559, acc: 50.00%] [G acc: 1.000000, mse: 0.169465]
4 [D loss: 0.639310, acc: 100.00%] [G acc: 0.000000, mse: 0.108334]
5 [D loss: 0.472161, acc: 93.75%] [G acc: 0.000000, mse: 0.088753]
6 [D loss: 0.362872, acc: 85.94%] [G acc: 0.000000, mse: 0.079219]
7 [D loss: 0.308999, acc: 85.94%] [G acc: 0.000000, mse: 0.087508]
8 [D loss: 0.286450, acc: 90.62%] [G acc: 0.000000, mse: 0.078368]
9 [D loss: 0.293211, acc: 87.50%] [G acc: 0.000000, mse: 0.081872]
10 [D loss: 0.301875, acc: 89.06%] [G acc: 0.000000, mse: 0.084327]
11 [D loss: 0.300903, acc: 79.69%] [G acc: 0.000000, mse: 0.084026]
12 [D loss: 0.277012, acc: 87.50%] [G acc: 0.000000, mse: 0.083081]
13 [D loss: 0.275925, acc: 87.50%] [G acc: 0.000000, mse: 0.076799]
14 [D loss: 0.271516, acc: 87.50%] [G acc: 0.000000, mse: 0.082170]
15 [D

In [10]:
autoencoder.predict([X_train])

AssertionError: ignored